In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import math
import functools
import random
import time
import csv
import os

## Comparison of the ILP and the greedy algorithm for the FPTUMKP

In [1]:
def generate_instance(n, m, slots, slot_length, multiple): # multiple = TRUE if the each campaign should have multiple possible slots
    if multiple == False:
        s = np.random.choice([0, 1, 2], n) # single slot
    else:
        s = [sorted(random.sample([0,1,2], random.randint(1, 3))) for i in range(n)] # multiple slots
    p = np.full(n, slot_length) # processing time
    c = np.random.randint(low = 10, high = 100, size = n) # cost
    b = np.random.randint(low = 70, high = 200, size = n) # benefit
    C = np.random.randint(low = 0.4*m, high = 0.95*m, size = n)*c # maximum cost

    zippedList =  list(zip(s, p, c, b, C))
    campaigns = pd.DataFrame(zippedList, columns = ['starting time', 'processing time' , 'cost', 'benefit', 'maximum cost'])
    campaigns.index.names = ['name']
    # print("Dataframe : " , campaigns, sep='\n')

    # matrix with take-rates
    T = np.random.random_sample((m, n))
    
    return campaigns, T

In [3]:
def greedy(n, campaigns, take_rates):
    campaigns['ratio'] = campaigns['benefit'] / campaigns['cost']
    sorted_campaigns = campaigns.sort_values(by=['ratio'], ascending=False)
    sorted_campaigns['total cost'] = 0
    customers = pd.DataFrame(take_rates)
    customers['campaigns'] = [[] for _ in range(len(customers))] # list to store assigned campaigns
    customers['blocking times'] = [set() for _ in range(len(customers))] # list to store blocking times
    customers.index.names = ['costumer']
    profit  = 0
    
    for j in range(len(sorted_campaigns)):
        current_campaign = sorted_campaigns.index[j]
        sorted_customers = customers.sort_values(by=[current_campaign], ascending=False)
        start_time = sorted_campaigns.loc[current_campaign, 'starting time']
        end_time = sorted_campaigns.loc[current_campaign, 'starting time'] + sorted_campaigns.loc[current_campaign, 'processing time']
        for i in range(len(sorted_customers)):
            current_costumer = sorted_customers.index[i]
            if sorted_campaigns.loc[current_campaign, 'total cost'] + sorted_campaigns.loc[current_campaign, 'cost'] <= sorted_campaigns.loc[current_campaign, 'maximum cost']:
                if start_time not in customers.loc[current_costumer, 'blocking times']:
                    sorted_campaigns.loc[current_campaign, 'total cost'] = sorted_campaigns.loc[current_campaign, 'cost'] + sorted_campaigns.loc[current_campaign, 'total cost']
                    customers.loc[current_costumer, 'campaigns'].append(current_campaign)
                    customers.loc[current_costumer, 'blocking times'].update(range(start_time,end_time))
                    profit += (sorted_campaigns.loc[current_campaign, 'benefit'] * customers.loc[current_costumer, current_campaign]) - sorted_campaigns.loc[current_campaign, 'cost']
            else:
                break
                
    return profit

In [4]:
def optimal(n, m, slots, b, c, C, s, T):
    # create empty model
    model = gp.Model()

    # add decision variables
    x = model.addMVar((m,n), vtype=GRB.BINARY, name='x') # variable x_ij
    z = model.addMVar((m,n,3), vtype=GRB.BINARY, name = 'z') # variable z_ijl

    # set objective function
    model.setObjective((sum((b[j]*T[i][j] - c[j]) * x[i,j] for j in range(n) for i in range(m))), GRB.MAXIMIZE)

    # add constraints
    model.addConstrs(((sum(c[j]*x[i,j] for i in range(m))) <= C[j]) for j in range(n))

    model.addConstrs(((sum(z[i,j,l] for j in range(n))) <= 1) for i in range(m) for l in range(slots))

    model.addConstrs((z[i,j,l] == 0) for j in range(n) for i in range(m) for l in range(slots) if l != s[j])

    # solve model
    model.optimize()

    # get solution
    if model.SolCount > 0:
        profit = model.objVal
        
    return profit

In [7]:
n = 10 # number of campaign
slots = 3 # 3 time slots
slot_length = 15

cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)

# write solutions into file
with open('{}/comparison_fptumkp.csv'.format(path), mode = 'w') as file_profits:
    writer_profits = csv.writer(file_profits, delimiter = ';')
    writer_profits.writerow(['number of customers','greedy profit','greedy time', 'optimal profit', 'optimal time'])

    for m in [1000, 2500, 5000, 7500, 10000, 15000, 20000, 50000, 100000, 200000, 400000]:
        if m < 100000:
            number_of_instances = 10
        else:
            number_of_instances = 5
        sum_greedy = 0.0
        sum_optimal = 0.0
        for _ in range(number_of_instances):
            campaigns, take_rates = generate_instance(n, m, slots, slot_length, False)

            start_greedy = time.time()
            profit_greedy = greedy(n, campaigns, take_rates)
            end_greedy = time.time()-start_greedy
            sum_greedy += end_greedy

            start_optimal = time.time()
            profit_optimal = optimal(n, m, slots, campaigns["benefit"].tolist(), campaigns["cost"].tolist(), campaigns["maximum cost"].tolist(), campaigns["starting time"].tolist(), take_rates)
            end_optimal = time.time()-start_optimal
            sum_optimal += end_optimal

            writer_profits.writerow([m, profit_greedy, end_greedy, profit_optimal, end_optimal])
            
file_profits.close()

[180106 101140 174588  91728 186689  90099 123599 117730  92097 107170]
Dataframe : 
      processing time  cost  benefit  starting time  maximum cost
name                                                             
0                  15    13      142              1       2341378
1                  15    52      137              1       5259280
2                  15    11      126              0       1920468
3                  15    96      163              1       8805888
4                  15    95       93              0      17735455
5                  15    35      105              0       3153465
6                  15    92       74              0      11371108
7                  15    83      114              2       9771590
8                  15    23      111              0       2118231
9                  15    16       87              1       1714720
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to